*Extracting features of 2019 movies from Wikipedia*

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from tmdbv3api import TMDb
from tmdbv3api import Movie
# The tmdbv3api library in Python is an unofficial wrapper for The Movie Database (TMDb) API version 3. 
# It allows developers to easily interact with the TMDb API to retrieve information about movies, TV shows, and actors.

import warnings
warnings.filterwarnings('ignore')

In [3]:
link="https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
data_1=pd.read_html(link,header=0)[2]
data_2=pd.read_html(link,header=0)[3]
data_3=pd.read_html(link,header=0)[4]
data_4=pd.read_html(link,header=0)[5]


In [4]:
data_1.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,J A N U A R Y,4,Rust Creek,IFC Films / Lunacy Productions,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,J A N U A R Y,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,J A N U A R Y,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,J A N U A R Y,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]


In [5]:
data = pd.concat([data_1, data_2, data_3, data_4], ignore_index=True)
data

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,J A N U A R Y,4,Rust Creek,IFC Films / Lunacy Productions,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,J A N U A R Y,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,J A N U A R Y,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,J A N U A R Y,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
241,D E C E M B E R,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
242,D E C E M B E R,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[225]
243,D E C E M B E R,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[226]
244,D E C E M B E R,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[227]


*As it is observed in the above table, we do not have the genre column so we will get the genre data from IMDb. We will need to create an API key in order to access the API key.*

In [6]:
tmdb=TMDb()
tmdb.api_key='dc6d032ccd5a234bc6937dd5e933075a'

In [7]:
tmdb_movie=Movie()
def get_genre(x):
    genres=[]
    result=tmdb_movie.search(x) #the title will be searched in the tmdb_movie
    movie_id = result[0].id #we will match the "id" with the "title"
    response = requests.get('https://api.themoviedb.org/3/movie/{}'.format(movie_id),params={'api_key':'dc6d032ccd5a234bc6937dd5e933075a'}) #we will get the result from the IMDb data
    data_json = response.json() #we will then convert it to a json file
    if data_json['genres']: #in the json file we will only need to extract the "genre"
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name']) #we will then add the "genre" to the empty genre list we created above
        return genre_str.join(genres)
    else:
        np.NaN # we will return the results but if we don't find anything we will consider it as a missing value


In [ ]:
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))
data

In [ ]:
data_2019_backup=data.copy(deep=True)
data_2019_backup.to_csv("data_2019_backup.csv")

In [9]:
data=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Code\\data_2019_backup.csv")

In [10]:
data_2019=data[['Title','Cast and crew','genres']]
data_2019

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama
...,...,...,...
241,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family
242,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance
243,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Action Drama
244,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History


*Extracting Directors*

In [11]:
def get_directors(x):
    if " (director)" in x:
        if " (director/screenplay)" in x:
            return x.split(" (director/screenplay)")[0]
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (director);" in x:
        return x.split(" (director); ")[0]
    elif " (director/\u200bscreenplay);" in x:
        return x.split(" (director/\u200bscreenplay);")[0]
    elif " (directors/screenplay);" in x:
        return x.split(" (directors/screenplay)")[0]
    elif " (directors/screenplay)" in x:
        return x.split(" (directors/screenplay)")[0]
    else:
        return x.split(" (director/screenplay)")[0]
    

In [12]:
data_2019['director_name']=data_2019['Cast and crew'].map(lambda x : get_directors(str(x)))

In [13]:
data_2019

,Title,Cast and crew,genres,director_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime,Jen McGowan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger
...,...,...,...,...
241,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane"
242,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig
243,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Action Drama,Sam Mendes
244,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton


In [26]:
data_2019['director_name'][92]=data_2019['director_name'][92].split(" (director); ")[0]
data_2019['director_name'][83]=data_2019['director_name'][83].split(" (director, screenplay);")[0]
data_2019['director_name'][89]=data_2019['director_name'][89].split(" (director, screenplay);")[0]


In [27]:

data_2019[data_2019['director_name'].str.contains('director')]

,Title,Cast and crew,genres,director_name


*Actor 1*

In [28]:
def get_actor_1(x):
    return (x.split(',')[-1])

In [29]:
data_2019['actor_1_name']=data_2019['Cast and crew'].map(lambda x : get_actor_1(x))
data_2019

,Title,Cast and crew,genres,director_name,actor_1_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel,Yorick van Wageningen
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime,Jen McGowan,Micah Hauptman
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Paul Braunstein
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Wes Studi
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Nicole Kidman
...,...,...,...,...,...
241,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Masi Oka
242,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig,Chris Cooper
243,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Action Drama,Sam Mendes,Benedict Cumberbatch
244,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,C.J. LeBlanc


*Actor 2*

In [30]:
def get_actor_2(x):
    if len(x.split(','))<2:
        return np.nan
    return (x.split(',')[-2])

In [31]:
data_2019['actor_2_name']=data_2019['Cast and crew'].map(lambda x : get_actor_2(x))
data_2019

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel,Yorick van Wageningen,Nik Dodani
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime,Jen McGowan,Micah Hauptman,Sean O'Bryan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Paul Braunstein,Oliver Dennis
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Wes Studi,Jonah Haur-King
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Nicole Kidman,Kevin Hart
...,...,...,...,...,...,...
241,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Masi Oka,DJ Khaled
242,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig,Chris Cooper,Louis Garrel
243,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Action Drama,Sam Mendes,Benedict Cumberbatch,Colin Firth
244,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,C.J. LeBlanc,O'Shea Jackson Jr.


Actor 3 

In [32]:
def get_actor_3(x):
    if len(x.split(','))<3:
        return np.nan
    return (x.split(',')[-3])

In [33]:
data_2019['actor_3_name']=data_2019['Cast and crew'].map(lambda x : get_actor_2(x))
data_2019

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel,Yorick van Wageningen,Nik Dodani,Nik Dodani
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime,Jen McGowan,Micah Hauptman,Sean O'Bryan,Sean O'Bryan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Paul Braunstein,Oliver Dennis,Oliver Dennis
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Wes Studi,Jonah Haur-King,Jonah Haur-King
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Nicole Kidman,Kevin Hart,Kevin Hart
...,...,...,...,...,...,...,...
241,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Masi Oka,DJ Khaled,DJ Khaled
242,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig,Chris Cooper,Louis Garrel,Louis Garrel
243,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Action Drama,Sam Mendes,Benedict Cumberbatch,Colin Firth,Colin Firth
244,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,C.J. LeBlanc,O'Shea Jackson Jr.,O'Shea Jackson Jr.


*Renaming The Column*

In [34]:
data_2019=data_2019.rename(columns={'Title':'movie_title'})

In [35]:
new_data_2019=data_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_data_2019

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Yorick van Wageningen,Nik Dodani,Nik Dodani,Horror Thriller Mystery,Escape Room
1,Jen McGowan,Micah Hauptman,Sean O'Bryan,Sean O'Bryan,Thriller Drama Action Crime,Rust Creek
2,Wilson Coneybeare,Paul Braunstein,Oliver Dennis,Oliver Dennis,Thriller,American Hangman
3,Charles Martin Smith,Wes Studi,Jonah Haur-King,Jonah Haur-King,Drama Adventure Family,A Dog's Way Home
4,Neil Burger,Nicole Kidman,Kevin Hart,Kevin Hart,Comedy Drama,The Upside
...,...,...,...,...,...,...
241,"Nick Bruno, Troy Quane",Masi Oka,DJ Khaled,DJ Khaled,Animation Action Adventure Comedy Family,Spies in Disguise
242,Greta Gerwig,Chris Cooper,Louis Garrel,Louis Garrel,Drama Romance,Little Women
243,Sam Mendes,Benedict Cumberbatch,Colin Firth,Colin Firth,War Action Drama,1917
244,Destin Daniel Cretton,C.J. LeBlanc,O'Shea Jackson Jr.,O'Shea Jackson Jr.,Drama Crime History,Just Mercy


In [36]:
new_data_2019['actor_2_name']=new_data_2019['actor_2_name'].replace(np.nan,'unknown')
new_data_2019['actor_3_name']=new_data_2019['actor_3_name'].replace(np.nan,'unknown')

In [37]:
new_data_2019.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [38]:
new_data_2019['movie_title']=new_data_2019['movie_title'].str.lower()

In [39]:
new_data_2019['comb']=new_data_2019['actor_1_name']+" "+new_data_2019['actor_2_name']+" "+new_data_2019['actor_3_name']+" "+new_data_2019['director_name']+" "+new_data_2019['genres']

In [40]:
new_data_2019.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Yorick van Wageningen,Nik Dodani,Nik Dodani,Horror Thriller Mystery,escape room,Yorick van Wageningen Nik Dodani Nik Dodani...
1,Jen McGowan,Micah Hauptman,Sean O'Bryan,Sean O'Bryan,Thriller Drama Action Crime,rust creek,Micah Hauptman Sean O'Bryan Sean O'Bryan Je...
2,Wilson Coneybeare,Paul Braunstein,Oliver Dennis,Oliver Dennis,Thriller,american hangman,Paul Braunstein Oliver Dennis Oliver Dennis...
3,Charles Martin Smith,Wes Studi,Jonah Haur-King,Jonah Haur-King,Drama Adventure Family,a dog's way home,Wes Studi Jonah Haur-King Jonah Haur-King C...
4,Neil Burger,Nicole Kidman,Kevin Hart,Kevin Hart,Comedy Drama,the upside,Nicole Kidman Kevin Hart Kevin Hart Neil Bu...


In [41]:
data_2018=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Processed Datasets\\data_3.csv")
data_2018.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Bruce Davison,Caitlin Gerard,Caitlin Gerard,Horror Mystery Thriller,insidious: the last key,Bruce Davison Caitlin Gerard Caitlin Gerard...
1,Christopher Radcliff,Tobias Campbell,Owen Campbell,Owen Campbell,Drama Mystery,the strange ones,Tobias Campbell Owen Campbell Owen Campbell...
2,Jaume Collet-Serra,Sam Neill,Jonathan Banks,Jonathan Banks,Action Thriller Mystery,the commuter,Sam Neill Jonathan Banks Jonathan Banks Jau...
3,Babak Najafi,Danny Glover,Billy Brown,Billy Brown,Thriller Action Crime,proud mary,Danny Glover Billy Brown Billy Brown Babak ...
4,Brett Donowho,Mike Epps,Sophia Bush,Sophia Bush,Action Crime Thriller,acts of violence,Mike Epps Sophia Bush Sophia Bush Brett Don...


In [42]:
old_data=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Processed Datasets\\new_data.csv")
old_data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,cch pounder joel david moore wes studi james c...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,johnny depp orlando bloom jack davenport gore ...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,christoph waltz rory kinnear stephanie sigman ...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,tom hardy christian bale joseph gordon-levitt ...
4,doug walker,doug walker,rob walker,unknown,documentary,star wars: episode vii - the force awakens ...,doug walker rob walker unknown doug walker doc...


In [43]:
my_data=pd.concat([data_2018,new_data_2019],ignore_index=True)
my_data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Bruce Davison,Caitlin Gerard,Caitlin Gerard,Horror Mystery Thriller,insidious: the last key,Bruce Davison Caitlin Gerard Caitlin Gerard...
1,Christopher Radcliff,Tobias Campbell,Owen Campbell,Owen Campbell,Drama Mystery,the strange ones,Tobias Campbell Owen Campbell Owen Campbell...
2,Jaume Collet-Serra,Sam Neill,Jonathan Banks,Jonathan Banks,Action Thriller Mystery,the commuter,Sam Neill Jonathan Banks Jonathan Banks Jau...
3,Babak Najafi,Danny Glover,Billy Brown,Billy Brown,Thriller Action Crime,proud mary,Danny Glover Billy Brown Billy Brown Babak ...
4,Brett Donowho,Mike Epps,Sophia Bush,Sophia Bush,Action Crime Thriller,acts of violence,Mike Epps Sophia Bush Sophia Bush Brett Don...
...,...,...,...,...,...,...,...
482,"Nick Bruno, Troy Quane",Masi Oka,DJ Khaled,DJ Khaled,Animation Action Adventure Comedy Family,spies in disguise,"Masi Oka DJ Khaled DJ Khaled Nick Bruno, Tr..."
483,Greta Gerwig,Chris Cooper,Louis Garrel,Louis Garrel,Drama Romance,little women,Chris Cooper Louis Garrel Louis Garrel Gret...
484,Sam Mendes,Benedict Cumberbatch,Colin Firth,Colin Firth,War Action Drama,1917,Benedict Cumberbatch Colin Firth Colin Firt...
485,Destin Daniel Cretton,C.J. LeBlanc,O'Shea Jackson Jr.,O'Shea Jackson Jr.,Drama Crime History,just mercy,C.J. LeBlanc O'Shea Jackson Jr. O'Shea Jack...


In [44]:
final_data=pd.concat([old_data,my_data],ignore_index=True)
final_data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,james cameron,cch pounder,joel david moore,wes studi,action adventure fantasy sci-fi,avatar,cch pounder joel david moore wes studi james c...
1,gore verbinski,johnny depp,orlando bloom,jack davenport,action adventure fantasy,pirates of the caribbean: at world's end,johnny depp orlando bloom jack davenport gore ...
2,sam mendes,christoph waltz,rory kinnear,stephanie sigman,action adventure thriller,spectre,christoph waltz rory kinnear stephanie sigman ...
3,christopher nolan,tom hardy,christian bale,joseph gordon-levitt,action thriller,the dark knight rises,tom hardy christian bale joseph gordon-levitt ...
4,doug walker,doug walker,rob walker,unknown,documentary,star wars: episode vii - the force awakens ...,doug walker rob walker unknown doug walker doc...
...,...,...,...,...,...,...,...
5930,"Nick Bruno, Troy Quane",Masi Oka,DJ Khaled,DJ Khaled,Animation Action Adventure Comedy Family,spies in disguise,"Masi Oka DJ Khaled DJ Khaled Nick Bruno, Tr..."
5931,Greta Gerwig,Chris Cooper,Louis Garrel,Louis Garrel,Drama Romance,little women,Chris Cooper Louis Garrel Louis Garrel Gret...
5932,Sam Mendes,Benedict Cumberbatch,Colin Firth,Colin Firth,War Action Drama,1917,Benedict Cumberbatch Colin Firth Colin Firt...
5933,Destin Daniel Cretton,C.J. LeBlanc,O'Shea Jackson Jr.,O'Shea Jackson Jr.,Drama Crime History,just mercy,C.J. LeBlanc O'Shea Jackson Jr. O'Shea Jack...


In [8]:
data=pd.read_csv("E:\\Data Science\\Project\\Netflix Recommendation System\\Processed Datasets\\main_data.csv",encoding='latin-1')

<IPython.core.display.Javascript object>

In [9]:
data.replace(np.nan,'unknown',inplace=True)

<IPython.core.display.Javascript object>

In [10]:
data.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [6]:
data.to_csv('main_data.csv',index=False)